In [1]:
# Install required Python packages

import sys
import subprocess

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

required_packages = [
    "numpy",
    "pandas",
    "astropy",
    "sgp4",
    "jupyter"
]

for pkg in required_packages:
    try:
        __import__(pkg)
        print(f"{pkg} already installed")
    except ImportError:
        print(f"Installing {pkg} ...")
        install(pkg)

print("\nAll required packages are installed.")


numpy already installed
pandas already installed
astropy already installed
sgp4 already installed
jupyter already installed

All required packages are installed.


Note: After running the installation cell for the first time, please restart the kernel and then run the notebook from top to bottom.

In [2]:
# Libraries and constants
import numpy as np
import pandas as pd

from sgp4.api import Satrec
from astropy import units as astro_u
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz
from astropy.coordinates import (
    GCRS,
    ITRS,
    CartesianRepresentation,
    EarthLocation
)

# Constants
r_earth = 6378.137      # Earth equatorial radius km
omega_e = 7.2921159e-5  # Earth rotation rate [rad/s]
mu_earth = 398600.4418  # Earth gravitational parameter [km^3/s^2]

In [ ]:
# TLE of the object
tle_line1 = "1 63223U 25052P   25244.59601767  .00010814  00000-0  51235-3 0  9991"
tle_line2 = "2 63223  97.4217 137.0451 0006365  74.2830 285.9107 15.19475170 25990"
# Create satellite object
sat = Satrec.twoline2rv(tle_line1, tle_line2)
# Extract TLE epoch (Julian Date) from SGP4 object
jd_epoch = sat.jdsatepoch
fr_epoch = sat.jdsatepochF

# Convert to Astropy Time object (UTC)
t_epoch = Time(jd_epoch + fr_epoch, format="jd", scale="utc")

In [4]:
# Define propagation time span
mean_motion = sat.no_kozai * 1440 / (2 * np.pi)  # rev/day
T_orbit = 86400 / mean_motion                   # seconds

# Time array: propagate for one full orbit
N = 500  # number of time steps
time_array = t_epoch + np.linspace(0, T_orbit, N) * astro_u.s


In [5]:
# Orbit propagation using SGP4 (TEME frame)

r_teme = np.zeros((N, 3))   # position km
v_teme = np.zeros((N, 3))   # velocity [km/s]


for k, t in enumerate(time_array):
    # Convert Astropy Time to Julian Date
    jd = t.jd
    fr = 0.0  # fractional part already included in jd

    # SGP4 propagation
    error_code, r, v = sat.sgp4(jd, fr)

    if error_code != 0:
        raise RuntimeError(f"SGP4 error at step {k}, code = {error_code}")

    r_teme[k, :] = r
    v_teme[k, :] = v


In [ ]:
# Space based Tracker orbital elements
tracker_epoch = Time("2025-09-01 00:00:00", scale="utc")

a = 6878.0              # km
e = 0.0
inc = np.deg2rad(97.4)
raan = np.deg2rad(72.628)
argp = np.deg2rad(331.7425)
M0 = np.deg2rad(0.0)

# Mean motion
n = np.sqrt(mu_earth / a**3)  # rad/s


In [18]:
def kepler_propagate(a, e, inc, raan, argp, M0, t, t0):
    """
    Two-body Kepler propagation (TEME-like inertial frame)
    """
    dt = (t - t0).to(astro_u.s).value
    M = M0 + n * dt

    # Circular orbit: E = M, nu = M
    nu = M
    r = a

    # Perifocal coordinates
    r_pqw = np.array([
        r * np.cos(nu),
        r * np.sin(nu),
        0.0
    ])

    v_pqw = np.array([
        -np.sqrt(mu_earth / a) * np.sin(nu),
         np.sqrt(mu_earth / a) * np.cos(nu),
         0.0
    ])

    # Rotation matrix PQW to TEME
    R3_W = np.array([
        [ np.cos(raan), -np.sin(raan), 0],
        [ np.sin(raan),  np.cos(raan), 0],
        [ 0, 0, 1]
    ])

    R1_i = np.array([
        [1, 0, 0],
        [0, np.cos(inc), -np.sin(inc)],
        [0, np.sin(inc),  np.cos(inc)]
    ])

    R3_w = np.array([
        [ np.cos(argp), -np.sin(argp), 0],
        [ np.sin(argp),  np.cos(argp), 0],
        [ 0, 0, 1]
    ])

    Q = R3_W @ R1_i @ R3_w

    r_eci = Q @ r_pqw
    v_eci = Q @ v_pqw

    return r_eci, v_eci


# Time window: EXACTLY one day from tracker epoch


dt = 5.0  # seconds (can refine later)
T_window = 24 * 3600  # one day in seconds

time_array_cross = tracker_epoch + np.arange(
    0, T_window + dt, dt
) *astro_u.s

In [19]:
# Crossing detection (corrected, one-day window)

FOV_HALF_ANGLE = np.deg2rad(15.0)
MAX_RANGE = 1000.0  # km

crossings = []
inside_fov = False
entry_time = None
min_range = np.inf

for t in time_array_cross:

    
    # Tracker state (Kepler)
    
    r_T, v_T = kepler_propagate(
        a, e, inc, raan, argp, M0, t, tracker_epoch
    )

    
    # Target state (SGP4)
    
    err, r_O, v_O = sat.sgp4(t.jd, 0.0)
    if err != 0:
        continue

    r_O = np.array(r_O)

    
    # Relative geometry
    
    r_rel = r_O - r_T
    rho = np.linalg.norm(r_rel)

    if rho == 0:
        continue

    b_hat = v_T / np.linalg.norm(v_T)
    r_hat = r_rel / rho

    cos_theta = np.clip(np.dot(b_hat, r_hat), -1.0, 1.0)
    theta = np.arccos(cos_theta)

    in_fov = (theta <= FOV_HALF_ANGLE) and (rho <= MAX_RANGE)

    
    # Event logic
    
    if in_fov:
        if not inside_fov:
            inside_fov = True
            entry_time = t
            min_range = rho
        else:
            min_range = min(min_range, rho)
    else:
        if inside_fov:
            crossings.append({
                "start_time_utc": entry_time.utc.iso,
                "end_time_utc": t.utc.iso,
                "duration_sec": (t - entry_time).to(astro_u.s).value,
                "min_range_km": min_range
            })
            inside_fov = False


if inside_fov:
    crossings.append({
        "start_time_utc": entry_time.utc.iso,
        "end_time_utc": time_array_cross[-1].utc.iso,
        "duration_sec": (
            time_array_cross[-1] - entry_time
        ).to(astro_u.s).value,
        "min_range_km": min_range
    })




In [20]:
from astropy.coordinates import get_sun

# Sunlight / eclipse test (Earth umbra)

def is_sunlit(r_obj_km, t):

    # Sun position in GCRS (km)
    sun = get_sun(t).cartesian.xyz.to(astro_u.km).value
    r_sun = np.array(sun)

    # Vector from object to Sun
    s = r_sun - r_obj_km

    # Condition 1: Earth between Sun and object
    if np.dot(r_obj_km, s) >= 0:
        return True  # Sun is not behind Earth

    # Condition 2: Distance from Earth–Sun line
    d_perp = np.linalg.norm(np.cross(r_obj_km, s)) / np.linalg.norm(s)

    if d_perp < r_earth:
        return False  # In Earth's shadow
    else:
        return True


In [21]:
# Crossing

crossings = []
inside_fov = False
entry_time = None
min_range = np.inf
sunlit_during_crossing = False

for t in time_array_cross:

    # Tracker state
    r_T, v_T = kepler_propagate(
        a, e, inc, raan, argp, M0, t, tracker_epoch
    )

    # Target state (SGP4)
    err, r_O, v_O = sat.sgp4(t.jd, 0.0)
    if err != 0:
        continue

    r_O = np.array(r_O)

    # Relative geometry
    r_rel = r_O - r_T
    rho = np.linalg.norm(r_rel)
    if rho == 0:
        continue

    # Camera boresight
    b_hat = v_T / np.linalg.norm(v_T)
    r_hat = r_rel / rho

    theta = np.arccos(
        np.clip(np.dot(b_hat, r_hat), -1.0, 1.0)
    )

    in_fov = (theta <= FOV_HALF_ANGLE) and (rho <= MAX_RANGE)

    if in_fov:
        if not inside_fov:
            inside_fov = True
            entry_time = t
            min_range = rho
            sunlit_during_crossing = False

        min_range = min(min_range, rho)

        # Sunlight check
        if is_sunlit(r_O, t):
            sunlit_during_crossing = True

    else:
        if inside_fov:
            crossings.append({
                "start_time_utc": entry_time.utc.iso,
                "end_time_utc": t.utc.iso,
                "duration_sec": (t - entry_time).to(astro_u.s).value,
                "min_range_km": min_range,
                "sunlit": sunlit_during_crossing,
                "detectable": sunlit_during_crossing
            })
            inside_fov = False
if inside_fov:
    crossings.append({
        "start_time_utc": entry_time.utc.iso,
        "end_time_utc": time_array_cross[-1].utc.iso,
        "duration_sec": (
            time_array_cross[-1] - entry_time
        ).to(astro_u.s).value,
        "min_range_km": min_range,  
        "sunlit": sunlit_during_crossing,
        "detectable": sunlit_during_crossing
    })



In [22]:
# Ground station 

gs_lat = 78.9066     # deg
gs_lon = 11.88916    # deg
gs_alt = 380.0       # meters

ground_station = EarthLocation(
    lat=gs_lat *astro_u.deg,
    lon=gs_lon *astro_u.deg,
    height=gs_alt *astro_u.m
)

# Field of regard (half-angle)
FOR_HALF_ANGLE = np.deg2rad(70.0 / 2.0)   # radians

def station_night(station, t):
    sun_altaz = get_sun(t).transform_to(
        AltAz(obstime=t, location=station)
    )
    return sun_altaz.alt.deg < 0.0

In [23]:
# Ground-based crossing detection
ground_crossings = []
inside_for = False
entry_time = None
min_range = np.inf
sunlit_during_crossing = False
night_during_crossing = False

for t in time_array_cross:

    # Target propagation (SGP4)
    err, r_O, v_O = sat.sgp4(t.jd, 0.0)
    if err != 0:
        continue

    # Target position as Astropy coordinate
    target_coord = GCRS(
        CartesianRepresentation(
            r_O[0] *astro_u.km,
            r_O[1] *astro_u.km,
            r_O[2] *astro_u.km
        ),
        obstime=t
    )

    # Convert to topocentric frame
    altaz = target_coord.transform_to(
        AltAz(obstime=t, location=ground_station)
    )

    elevation = altaz.alt.to(astro_u.rad).value
    azimuth = altaz.az.deg

    # Field of regard condition (cone about zenith)
    in_for = elevation >= (np.pi/2 - FOR_HALF_ANGLE)

    # Range
    rho = altaz.distance.to(astro_u.km).value

    if in_for:
        if not inside_for:
            inside_for = True
            entry_time = t
            min_range = rho
            sunlit_during_crossing = False
            night_during_crossing = False

        min_range = min(min_range, rho)

        # Visibility conditions
        if is_sunlit(np.array(r_O), t):
            sunlit_during_crossing = True

        if station_night(ground_station, t):
            night_during_crossing = True

    else:
        if inside_for:
            ground_crossings.append({
                "start_time_utc": entry_time.utc.iso,
                "end_time_utc": t.utc.iso,
                "duration_sec": (t - entry_time).to(astro_u.s).value,
                "min_range_km": min_range,
                "sunlit": sunlit_during_crossing,
                "station_night": night_during_crossing,
                "detectable": sunlit_during_crossing and night_during_crossing
            })
            inside_for = False
if inside_for:
    ground_crossings.append({
        "start_time_utc": entry_time.utc.iso,
        "end_time_utc": time_array_cross[-1].utc.iso,
        "duration_sec": (time_array_cross[-1] - entry_time).to(astro_u.s).value,
        "min_range_km": min_range,
        "sunlit": sunlit_during_crossing,
        "station_night": night_during_crossing,
        "detectable": sunlit_during_crossing and night_during_crossing
    })

In [24]:
# Combined table
df_space = pd.DataFrame(crossings)
df_space["tracker_type"] = "space_based"

df_ground = pd.DataFrame(ground_crossings)
df_ground["tracker_type"] = "ground_based"

df_combined = pd.concat(
    [df_space, df_ground],
    ignore_index=True
)

df_combined.to_csv(
    "space_and_ground_results.csv",
    index=False
)

df_combined.head()

print("Results saved to 'space_and_ground_results.csv'")

Results saved to 'space_and_ground_results.csv'
